# GLITCH
### 1. PREP DATA:
- Downloads news headlines asynchronously (with timestamp) + [timestamp, close_price, volume] klines in high frequency (1sec, 1min)
- TextBlob NLP scores them
### 2. IDENTIFY TRENDS:
- Define a trend: when subsequent prices keep going either up or down (could use the pivot system in V3)
- Measure continuity of trends: start at the news timestamp, track the trend and calculate 4 types of impacts (short, medium, long, very long - 15 minute, 1 hour, 1 day, 3 days). Add layers of impact if impact score is bigger
### 3. NORMALIZE TREND IMPACT:
- Normalize the impact as:
- Example:
    - News headline starts at t = 0.
    - Price increases by 5% over 300 seconds before reversing.
	- Average seconds per 1% =  300 \ 5% = 60 seconds for every 1%.
### 4. AGGREGATE DATA:
- Group by importance score ranges (e.g., 0-20, 21-40, etc.).
- Calculate the average seconds/1% impact for each group.
### 5. REGRESSIVE MODEL:
- Predicts impact of next upcoming headlines:
    - Input: News importance score, sentiment, entity, sector, etc.
    - Output: Average seconds per 1% importance.
<br>_Try KNIME regression models_<br>
### 6. CALCULATE TOTAL IMPACT DURATION
- For the latest piece of news:
- Use the importance percent score (e.g., 30% for significant news) and the predicted seconds per 1% impact (e.g., 7000 seconds per 1%).
- Compute total duration:

Total Impact Duration (seconds) = Importance Score (\%) \* Seconds per 1\% Impact

Example:
- Importance score = 30%.
- Seconds per 1% = 7000.
- Total impact duration =  30 \* 7000 = 210,000 seconds (~58 hours).

### 7. TREND-BASED ACTION
1. Good News (Uptrend)
	- Action: Buy immediately.
	- Estimate End of Trend:
	- Add the total impact duration (calculated in Step 1) to the news timestamp.
	- Monitor price movements within the predicted trend window.
2. Bad News (Downtrend)
	- Action: Sell immediately.
	- Estimate End of Trend:
	- Same approach as for uptrend but anticipate a downward price movement.

### 8. REAL-TIME MONITORING
- After taking the action (buy/sell):
- Continuously track the price.
- Compare the observed trend duration against the predicted duration.
- If the trend reverses significantly before the predicted duration (e.g., due to unexpected market events), re-evaluate the position.

### 9. ERROR HANDLING and ADJUSTMENTS
- Unexpected Reversals:
	- Include a stop-loss or trailing stop mechanism to limit losses if the trend reverses unexpectedly.
- Trend Strength:
	- Use metrics like momentum indicators or volume spikes to confirm trend direction.

### 10. BACKTESTING and VALIDATION
- Simulate historical data to evaluate:
	-	Prediction accuracy for trend direction.
	-	Success rate for timing the start and end of trends.
	-	Profitability of the buy/sell actions.

### 11. OPTIMIZE FURTHER
- Incorporate reinforcement learning or adaptive models to improve predictions of:
	-	Seconds per 1% impact.
	-	Trend direction classification.

# Possible useful data
[This](https://paperswithbacktest.com/dataset/all-daily-news) link to Papers with Backtest, showing overtime sentiment analysis of companies/cryptos

# Imports/Paths/Dependencies

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
PATH_ROOT = '/home/krisskaron/GLITCH_REBORN/__WAR__'
PATH_NEWS = f'{PATH_ROOT}/NEWS.csv'

sys.path.append(f'{PATH_ROOT}/APIs')
sys.path.append(f'{PATH_ROOT}/class')
from AMPLITUDE import *

/home/krisskaron/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


--------------------------------------------------------------------

# Amplitude, Period, Decay (from news)

In [2]:
from _impact import Impactor

input_file = "/home/krisskaron/GLITCH_REBORN/__WAR__/NEWS.csv"
output_file = "/home/krisskaron/GLITCH_REBORN/__WAR__/csv/NEWS_IMPACT.csv"

impactor = Impactor(input_file, output_file)
impact_df = impactor.run()

print(impact_df.head())  # Display the first few rows of the output

Successfully loaded data from /home/krisskaron/GLITCH_REBORN/__WAR__/NEWS.csv
Impact data saved to /home/krisskaron/GLITCH_REBORN/__WAR__/csv/NEWS_IMPACT.csv
    assessed_news_headline_id  impact news_headline_publication_timestamp  \
128                     925.0   10.70           2024-02-09 12:00:00+00:00   
169                     950.0   11.01           2024-02-14 12:00:00+00:00   
185                     875.0   10.08           2024-02-16 12:00:00+00:00   
271                     869.0   10.01           2024-02-27 06:00:00+00:00   
288                     992.0   11.53           2024-02-29 12:00:00+00:00   

     insignificance_timestamp  period_minutes  
128 2024-02-09 12:06:00+00:00             6.0  
169 2024-02-14 12:09:00+00:00             9.0  
185 2024-02-16 12:00:00+00:00             0.0  
271 2024-02-27 06:00:00+00:00             0.0  
288 2024-02-29 12:14:00+00:00            14.0  


-----------

# REINFORCEMENT LEARNING

In [3]:
from _RL import HarveySpecter
from stable_baselines3 import PPO, SAC

env = HarveySpecter()
model = PPO("MlpPolicy", env, learning_rate=0.0001, verbose=1)
model.learn(total_timesteps=100000)
model.save("trend_prediction_model")

2025-01-28 18:50:52.341173: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 18:50:52.732285: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 18:50:53.690028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/krisskaron/.local/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/krisskaron/.local/lib/python3.10/site-packages/gymnasium/s

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/krisskaron/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 340      |
|    ep_rew_mean     | 2.8e+04  |
| time/              |          |
|    fps             | 46       |
|    iterations      | 1        |
|    time_elapsed    | 44       |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 541           |
|    ep_rew_mean          | 3.8e+04       |
| time/                   |               |
|    fps                  | 46            |
|    iterations           | 2             |
|    time_elapsed         | 88            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.9751664e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.42         |
|    explained_variance   | -3.16e-05     |


In [4]:
from stable_baselines3 import PPO

model_path = "/home/krisskaron/GLITCH_REBORN/__WAR__/trend_prediction_model.zip"
model = PPO.load(model_path)
print(model.policy)
print("Training Log:")
print(model.get_parameters())

from stable_baselines3.common.evaluation import evaluate_policy
env = HarveySpecter()
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)
print(f"Mean reward: {mean_reward}, Standard deviation: {std_reward}")

/home/krisskaron/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)
Training Log:
{'policy': OrderedDict([('log_std', tensor([-0.0240], device='cuda:0')), ('mlp_extractor.policy_net.0.weight', tensor([[

/home/krisskaron/.local/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/krisskaron/.local/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/krisskaron/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 49165.30835920989, Standard deviation: 30715.528699802955


# NEWS_PIVOTS

In [5]:
# File paths
PATH_ROOT = '/home/krisskaron/GLITCH_REBORN/__WAR__'
PATH_PIVOT_PERIODS = f'{PATH_ROOT}/csv/PIVOT_PERIODS_1m.csv'
PATH_NEWS_IMPACT = f'{PATH_ROOT}/csv/NEWS_IMPACT.csv'

# Load data with timestamp parsing
pivot_df = pd.read_csv(PATH_PIVOT_PERIODS, parse_dates=['buy_time', 'sell_time'])
news_df = pd.read_csv(PATH_NEWS_IMPACT, parse_dates=['news_headline_publication_timestamp', 'insignificance_timestamp'])

# Remove timezone information for compatibility
pivot_df['buy_time'] = pivot_df['buy_time'].dt.tz_localize(None)
pivot_df['sell_time'] = pivot_df['sell_time'].dt.tz_localize(None)
news_df['news_headline_publication_timestamp'] = news_df['news_headline_publication_timestamp'].dt.tz_localize(None)
news_df['insignificance_timestamp'] = news_df['insignificance_timestamp'].dt.tz_localize(None)


def adjust_news_impact_with_model(model):
    adjusted_news = []
    for _, row in news_df.iterrows():
        state = np.array([
            row['impact'] / 100,  # Normalize impact
            0.5,  # Placeholder time difference ratio
            0.5  # Placeholder normalized period
        ], dtype=np.float32)
        action, _ = model.predict(state)
        new_impact = max(0, row['impact'] + action[0])  # Ensure non-negative impact

        # Calculate new insignificance timestamp
        decay_factor = 0.01
        new_period_minutes = int(np.log(10 / max(new_impact, 1e-6)) / -decay_factor)
        new_insignificance_timestamp = row['news_headline_publication_timestamp'] + timedelta(minutes=new_period_minutes)
        adjusted_news.append({
            'assessed_news_headline_id': row['assessed_news_headline_id'],
            'impact': new_impact,
            'news_headline_publication_timestamp': row['news_headline_publication_timestamp'],
            'insignificance_timestamp': new_insignificance_timestamp,
            'period_minutes': new_period_minutes  # Store the calculated period in minutes
        })
    adjusted_df = pd.DataFrame(adjusted_news)
    adjusted_df.to_csv(f'{PATH_ROOT}/csv/NEWS_PERIODS.csv', index=False)
    print("Adjusted news impact saved.")

adjust_news_impact_with_model(model)

Adjusted news impact saved.
